In [6]:
import sys
sys.path.append('../')
from dataset import get_activitynet_dataset, get_kineticslike_dataset
from dataset import get_son2parent, get_emb, get_dataloader
from metric import Metric
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

dataset_name, emb_type = 'activitynet', 'cone'

if dataset_name == 'activitynet':
    tree_file = '../activity_net_depth_v5.csv'
    if emb_type == 'cone' : emb_file = '../cone_activity_net300.pth' 
    if emb_type == 'glove' : emb_file = '../w2v_activity_300d.pth'
    feat_path, anno_fn = '../data.pickle', '../activity_net.v1-3.json'
    Xtr, Xval, ytr, yval, label_set = get_activitynet_dataset(feat_path, anno_fn)
    
elif dataset_name == 'kinetics':
    tree_file = '../kinetics_depth_v2.csv'
    if emb_type == 'cone' : emb_file = '../cones_kinetics300.pth'
    if emb_type == 'glove' : emb_file = '../w2v_kinetics_300d.pth'
    train_pth_path, valid_pth_path = '../kinetics_train.pth', '../kinetics_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)
    
elif dataset_name == 'moments':
    tree_file = '../moments_depth_v4.csv'
    if emb_type == 'cone' : emb_file = '../cones_moments300.pth'
    if emb_type == 'glove' : emb_file = '../w2v_moments_300d.pth'
    train_pth_path, valid_pth_path = '../moments_train.pth', '../moments_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)

son2parent = get_son2parent(tree_file)
emb = get_emb('hyp', emb_file, label_set)
train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
metric = Metric(label_set, son2parent)

### T=0.1不行

In [7]:
from pmath import pair_wise_cos, pair_wise_eud, pair_wise_hyp
from model import RegressNet
import torch
c, T, epochs, dist_func, eval_dist = .1,.1, 50, pair_wise_hyp, pair_wise_cos
torch.manual_seed(42)
emb = emb.cuda()

train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
model = RegressNet(T, c, dist_func, eval_dist, train_loader, val_loader, emb, metric)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model._train(optimizer, epochs, T, eval_interval = 5)

loss:1.291,acc:0.669,1hop_acc:0.829,2hop_acc:0.908,mAP:0.584,1hop_mAP:0.767,2hop_mAP:0.871
mAP:0.620,1hop_mAP:0.958,2hop_mAP:0.983
loss:1.185,acc:0.711,1hop_acc:0.838,2hop_acc:0.908,mAP:0.623,1hop_mAP:0.791,2hop_mAP:0.885
mAP:0.712,1hop_mAP:0.955,2hop_mAP:0.979
loss:1.226,acc:0.716,1hop_acc:0.837,2hop_acc:0.909,mAP:0.636,1hop_mAP:0.798,2hop_mAP:0.889
mAP:0.720,1hop_mAP:0.960,2hop_mAP:0.982
loss:1.216,acc:0.726,1hop_acc:0.849,2hop_acc:0.914,mAP:0.643,1hop_mAP:0.802,2hop_mAP:0.891
mAP:0.730,1hop_mAP:0.960,2hop_mAP:0.981
loss:1.272,acc:0.726,1hop_acc:0.849,2hop_acc:0.911,mAP:0.645,1hop_mAP:0.803,2hop_mAP:0.890
mAP:0.730,1hop_mAP:0.962,2hop_mAP:0.981
loss:1.268,acc:0.734,1hop_acc:0.849,2hop_acc:0.915,mAP:0.651,1hop_mAP:0.805,2hop_mAP:0.892
mAP:0.740,1hop_mAP:0.960,2hop_mAP:0.981
loss:1.262,acc:0.733,1hop_acc:0.847,2hop_acc:0.913,mAP:0.652,1hop_mAP:0.806,2hop_mAP:0.893
mAP:0.740,1hop_mAP:0.959,2hop_mAP:0.981
loss:1.299,acc:0.720,1hop_acc:0.840,2hop_acc:0.912,mAP:0.653,1hop_mAP:0.803,2hop_mA

### T= 1才行

In [8]:
from pmath import pair_wise_cos, pair_wise_eud, pair_wise_hyp
from model import RegressNet
import torch
c, T, epochs, dist_func, eval_dist = .1,1, 50, pair_wise_hyp, pair_wise_cos
torch.manual_seed(42)
emb = emb.cuda()

train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
model = RegressNet(T, c, dist_func, eval_dist, train_loader, val_loader, emb, metric)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model._train(optimizer, epochs, T, eval_interval = 5)

loss:3.328,acc:0.635,1hop_acc:0.821,2hop_acc:0.898,mAP:0.568,1hop_mAP:0.811,2hop_mAP:0.895
mAP:0.595,1hop_mAP:0.977,2hop_mAP:0.988
loss:3.025,acc:0.731,1hop_acc:0.858,2hop_acc:0.920,mAP:0.646,1hop_mAP:0.847,2hop_mAP:0.913
mAP:0.752,1hop_mAP:0.965,2hop_mAP:0.981
loss:2.968,acc:0.744,1hop_acc:0.858,2hop_acc:0.919,mAP:0.667,1hop_mAP:0.849,2hop_mAP:0.913
mAP:0.778,1hop_mAP:0.960,2hop_mAP:0.977
loss:2.945,acc:0.749,1hop_acc:0.859,2hop_acc:0.919,mAP:0.675,1hop_mAP:0.847,2hop_mAP:0.911
mAP:0.788,1hop_mAP:0.955,2hop_mAP:0.974
loss:2.934,acc:0.751,1hop_acc:0.858,2hop_acc:0.918,mAP:0.678,1hop_mAP:0.844,2hop_mAP:0.909
mAP:0.790,1hop_mAP:0.951,2hop_mAP:0.972
loss:2.925,acc:0.750,1hop_acc:0.857,2hop_acc:0.918,mAP:0.679,1hop_mAP:0.842,2hop_mAP:0.908
mAP:0.791,1hop_mAP:0.948,2hop_mAP:0.971
loss:2.917,acc:0.750,1hop_acc:0.856,2hop_acc:0.918,mAP:0.679,1hop_mAP:0.840,2hop_mAP:0.908
mAP:0.792,1hop_mAP:0.945,2hop_mAP:0.969
loss:2.910,acc:0.751,1hop_acc:0.857,2hop_acc:0.918,mAP:0.680,1hop_mAP:0.840,2hop_mA

### Step2 换成200D 三个数据集都有

In [1]:
import sys
sys.path.append('../')
from dataset import get_activitynet_dataset, get_kineticslike_dataset
from dataset import get_son2parent, get_emb, get_dataloader
from metric import Metric
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

dataset_name, emb_type = 'activitynet', 'cone'

if dataset_name == 'activitynet':
    tree_file = '../activity_net_depth_v5.csv'
    if emb_type == 'cone' : emb_file = '../cone_activity_net200.pth' 
    if emb_type == 'glove' : emb_file = '../w2v_activity_300d.pth'
    feat_path, anno_fn = '../data.pickle', '../activity_net.v1-3.json'
    Xtr, Xval, ytr, yval, label_set = get_activitynet_dataset(feat_path, anno_fn)
    
elif dataset_name == 'kinetics':
    tree_file = '../kinetics_depth_v2.csv'
    if emb_type == 'cone' : emb_file = '../cones_kinetics300.pth'
    if emb_type == 'glove' : emb_file = '../w2v_kinetics_300d.pth'
    train_pth_path, valid_pth_path = '../kinetics_train.pth', '../kinetics_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)
    
elif dataset_name == 'moments':
    tree_file = '../moments_depth_v4.csv'
    if emb_type == 'cone' : emb_file = '../cones_moments300.pth'
    if emb_type == 'glove' : emb_file = '../w2v_moments_300d.pth'
    train_pth_path, valid_pth_path = '../moments_train.pth', '../moments_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)

son2parent = get_son2parent(tree_file)
emb = get_emb('hyp', emb_file, label_set)
train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
metric = Metric(label_set, son2parent)

In [2]:
from pmath import pair_wise_cos, pair_wise_eud, pair_wise_hyp
from model import RegressNet
import torch
c, T, epochs, dist_func, eval_dist = .1,1, 50, pair_wise_hyp, pair_wise_cos
torch.manual_seed(42)
emb = emb.cuda()

train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
model = RegressNet(T, c, dist_func, eval_dist, train_loader, val_loader, emb, metric)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model._train(optimizer, epochs, T, eval_interval = 5)

loss:3.318,acc:0.649,1hop_acc:0.822,2hop_acc:0.900,mAP:0.571,1hop_mAP:0.812,2hop_mAP:0.896
mAP:0.608,1hop_mAP:0.974,2hop_mAP:0.986
loss:3.034,acc:0.729,1hop_acc:0.858,2hop_acc:0.917,mAP:0.647,1hop_mAP:0.847,2hop_mAP:0.913
mAP:0.756,1hop_mAP:0.966,2hop_mAP:0.981
loss:2.980,acc:0.742,1hop_acc:0.860,2hop_acc:0.919,mAP:0.666,1hop_mAP:0.850,2hop_mAP:0.914
mAP:0.778,1hop_mAP:0.961,2hop_mAP:0.978
loss:2.955,acc:0.748,1hop_acc:0.858,2hop_acc:0.918,mAP:0.674,1hop_mAP:0.848,2hop_mAP:0.911
mAP:0.787,1hop_mAP:0.955,2hop_mAP:0.974
loss:2.942,acc:0.749,1hop_acc:0.859,2hop_acc:0.918,mAP:0.676,1hop_mAP:0.845,2hop_mAP:0.909
mAP:0.789,1hop_mAP:0.952,2hop_mAP:0.972
loss:2.933,acc:0.747,1hop_acc:0.858,2hop_acc:0.916,mAP:0.678,1hop_mAP:0.843,2hop_mAP:0.908
mAP:0.789,1hop_mAP:0.950,2hop_mAP:0.971
loss:2.925,acc:0.749,1hop_acc:0.858,2hop_acc:0.916,mAP:0.679,1hop_mAP:0.843,2hop_mAP:0.907
mAP:0.789,1hop_mAP:0.948,2hop_mAP:0.970
loss:2.921,acc:0.750,1hop_acc:0.858,2hop_acc:0.917,mAP:0.679,1hop_mAP:0.842,2hop_mA

In [4]:
import sys
sys.path.append('../')
from dataset import get_activitynet_dataset, get_kineticslike_dataset
from dataset import get_son2parent, get_emb, get_dataloader
from metric import Metric
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

dataset_name, emb_type = 'kinetics', 'cone'

if dataset_name == 'activitynet':
    tree_file = '../activity_net_depth_v5.csv'
    if emb_type == 'cone' : emb_file = '../cone_activity_net200.pth' 
    if emb_type == 'glove' : emb_file = '../w2v_activity_300d.pth'
    feat_path, anno_fn = '../data.pickle', '../activity_net.v1-3.json'
    Xtr, Xval, ytr, yval, label_set = get_activitynet_dataset(feat_path, anno_fn)
    
elif dataset_name == 'kinetics':
    tree_file = '../kinetics_depth_v2.csv'
    if emb_type == 'cone' : emb_file = '../cone_kinetics_200.pth'
    if emb_type == 'glove' : emb_file = '../w2v_kinetics_300d.pth'
    train_pth_path, valid_pth_path = '../kinetics_train.pth', '../kinetics_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)
    
elif dataset_name == 'moments':
    tree_file = '../moments_depth_v4.csv'
    if emb_type == 'cone' : emb_file = '../cones_moments300.pth'
    if emb_type == 'glove' : emb_file = '../w2v_moments_300d.pth'
    train_pth_path, valid_pth_path = '../moments_train.pth', '../moments_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)

son2parent = get_son2parent(tree_file)
emb = get_emb('hyp', emb_file, label_set)
train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
metric = Metric(label_set, son2parent)

In [5]:
from pmath import pair_wise_cos, pair_wise_eud, pair_wise_hyp
from model import RegressNet
import torch
c, T, epochs, dist_func, eval_dist = .1,1, 50, pair_wise_hyp, pair_wise_cos
torch.manual_seed(42)
emb = emb.cuda()

train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
model = RegressNet(T, c, dist_func, eval_dist, train_loader, val_loader, emb, metric)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model._train(optimizer, epochs, T, eval_interval = 5)

loss:2.935,acc:0.762,1hop_acc:0.872,2hop_acc:0.914,mAP:0.573,1hop_mAP:0.844,2hop_mAP:0.894
mAP:0.698,1hop_mAP:0.956,2hop_mAP:0.969
loss:2.845,acc:0.770,1hop_acc:0.870,2hop_acc:0.913,mAP:0.595,1hop_mAP:0.834,2hop_mAP:0.887
mAP:0.719,1hop_mAP:0.937,2hop_mAP:0.957
loss:2.846,acc:0.771,1hop_acc:0.870,2hop_acc:0.912,mAP:0.594,1hop_mAP:0.827,2hop_mAP:0.882
mAP:0.717,1hop_mAP:0.929,2hop_mAP:0.951
loss:2.850,acc:0.769,1hop_acc:0.867,2hop_acc:0.911,mAP:0.593,1hop_mAP:0.824,2hop_mAP:0.880
mAP:0.715,1hop_mAP:0.928,2hop_mAP:0.950
loss:2.854,acc:0.762,1hop_acc:0.864,2hop_acc:0.908,mAP:0.590,1hop_mAP:0.821,2hop_mAP:0.878
mAP:0.713,1hop_mAP:0.927,2hop_mAP:0.949
loss:2.853,acc:0.763,1hop_acc:0.863,2hop_acc:0.906,mAP:0.590,1hop_mAP:0.822,2hop_mAP:0.878
mAP:0.712,1hop_mAP:0.926,2hop_mAP:0.949
loss:2.856,acc:0.762,1hop_acc:0.863,2hop_acc:0.908,mAP:0.589,1hop_mAP:0.819,2hop_mAP:0.876
mAP:0.711,1hop_mAP:0.924,2hop_mAP:0.947
loss:2.854,acc:0.763,1hop_acc:0.860,2hop_acc:0.903,mAP:0.589,1hop_mAP:0.816,2hop_mA

In [7]:
import sys
sys.path.append('../')
from dataset import get_activitynet_dataset, get_kineticslike_dataset
from dataset import get_son2parent, get_emb, get_dataloader
from metric import Metric
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

dataset_name, emb_type = 'moments', 'cone'

if dataset_name == 'activitynet':
    tree_file = '../activity_net_depth_v5.csv'
    if emb_type == 'cone' : emb_file = '../cone_activity_net200.pth' 
    if emb_type == 'glove' : emb_file = '../w2v_activity_300d.pth'
    feat_path, anno_fn = '../data.pickle', '../activity_net.v1-3.json'
    Xtr, Xval, ytr, yval, label_set = get_activitynet_dataset(feat_path, anno_fn)
    
elif dataset_name == 'kinetics':
    tree_file = '../kinetics_depth_v2.csv'
    if emb_type == 'cone' : emb_file = '../cone_kinetics_200.pth'
    if emb_type == 'glove' : emb_file = '../w2v_kinetics_300d.pth'
    train_pth_path, valid_pth_path = '../kinetics_train.pth', '../kinetics_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)
    
elif dataset_name == 'moments':
    tree_file = '../moments_depth_v5.csv'
    if emb_type == 'cone' : emb_file = '../cone_moments_200.pth'
    if emb_type == 'glove' : emb_file = '../w2v_moments_300d.pth'
    train_pth_path, valid_pth_path = '../moments_train.pth', '../moments_val.pth'
    Xtr, Xval, ytr, yval, label_set = get_kineticslike_dataset(train_pth_path, valid_pth_path)

son2parent = get_son2parent(tree_file)
emb = get_emb('hyp', emb_file, label_set)
train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
metric = Metric(label_set, son2parent)

In [8]:
from pmath import pair_wise_cos, pair_wise_eud, pair_wise_hyp
from model import RegressNet
import torch
c, T, epochs, dist_func, eval_dist = .1,1, 20, pair_wise_hyp, pair_wise_cos
torch.manual_seed(42)
emb = emb.cuda()

train_loader, val_loader = get_dataloader(Xtr, Xval, ytr, yval, emb, batch_size = 128)
model = RegressNet(T, c, dist_func, eval_dist, train_loader, val_loader, emb, metric)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model._train(optimizer, epochs, T, eval_interval = 2)

loss:4.406,acc:0.133,1hop_acc:0.188,2hop_acc:0.375,mAP:0.149,1hop_mAP:0.182,2hop_mAP:0.365
mAP:0.209,1hop_mAP:0.388,2hop_mAP:0.527
loss:4.365,acc:0.153,1hop_acc:0.207,2hop_acc:0.380,mAP:0.157,1hop_mAP:0.190,2hop_mAP:0.373
mAP:0.254,1hop_mAP:0.414,2hop_mAP:0.557
loss:4.353,acc:0.160,1hop_acc:0.214,2hop_acc:0.391,mAP:0.160,1hop_mAP:0.196,2hop_mAP:0.378
mAP:0.273,1hop_mAP:0.417,2hop_mAP:0.561
loss:4.352,acc:0.164,1hop_acc:0.218,2hop_acc:0.393,mAP:0.162,1hop_mAP:0.199,2hop_mAP:0.380
mAP:0.281,1hop_mAP:0.419,2hop_mAP:0.563
loss:4.356,acc:0.164,1hop_acc:0.219,2hop_acc:0.398,mAP:0.163,1hop_mAP:0.200,2hop_mAP:0.381
mAP:0.287,1hop_mAP:0.420,2hop_mAP:0.564
loss:4.358,acc:0.166,1hop_acc:0.222,2hop_acc:0.403,mAP:0.163,1hop_mAP:0.201,2hop_mAP:0.381
mAP:0.292,1hop_mAP:0.419,2hop_mAP:0.560
loss:4.366,acc:0.166,1hop_acc:0.222,2hop_acc:0.402,mAP:0.163,1hop_mAP:0.201,2hop_mAP:0.381
mAP:0.296,1hop_mAP:0.419,2hop_mAP:0.561
loss:4.370,acc:0.167,1hop_acc:0.223,2hop_acc:0.406,mAP:0.163,1hop_mAP:0.201,2hop_mA